<a href="https://colab.research.google.com/github/nerudxlf/get_directory/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

from google.colab import files

files.upload()

In [ ]:
omstu_full_df = pd.read_excel("2021-03-01_omstu.xlsx")
scopus_full_df = pd.read_excel("2021-03-01_scopus.xlsx")
wos_full_df = pd.read_excel("2021-03-01_wos.xlsx")
staff = pd.read_excel("staff.xlsx")

unnamed_list = ['Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7',
                'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13',
                'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19',
                'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22']

scopus_filter_id_author = scopus_full_df.filter(["id", "author name"])
wos_filter_id_author = wos_full_df.filter(["id", "name"])

omstu_join_scopus = pd.merge(left=omstu_full_df, right=scopus_filter_id_author, how="left",
                            left_on="scopus author id", right_on="id")

omstu_join_scopus_wos = pd.merge(left=omstu_join_scopus, right=wos_filter_id_author, how="left",
                                left_on="researcher id", right_on="id")

omstu_join_scopus_wos["ФИО"] = omstu_join_scopus_wos[omstu_join_scopus_wos.columns[0:3]].apply(
    lambda x: " ".join(x.dropna().astype(str)), axis=1)

omstu_join_scopus_wos.drop(['name_x', 'surname', 'patronymic', 'position', 'faculty', 'department', 'id_x', 'id_y'],
                          axis=1, inplace=True)

omstu_join_scopus_wos.rename(columns={"author name": "name in scopus", "name_y": "name in wos"}, inplace=True)

df_result_staff_and_omstu = pd.merge(left=staff, right=omstu_join_scopus_wos, how="left", left_on="Сотрудник",
                                    right_on="ФИО")

df_result_staff_and_omstu.drop(unnamed_list, axis=1, inplace=True)
df_result_staff_and_omstu.to_excel("result.xlsx", index=None)